In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv('data/ts_hr_feb_2020.csv.zip', parse_dates=['date'])
data.shape

(696, 2)

In [3]:
data.head()

,date,sales_value
0,2020-02-01 00:00:00,28.60
1,2020-02-01 01:00:00,122.84
2,2020-02-01 02:00:00,112.87
3,2020-02-01 03:00:00,357.20
4,2020-02-01 04:00:00,642.56


In [4]:
last_sunday = pd.to_datetime("2020-02-23")

train       = data[data['date'] < last_sunday]
val         = data[data['date'] >= last_sunday]

In [5]:
def prep_seqs(df, l=24, h=24):

    X = []
    Y = []

    for i in range(l, df.shape[0]-h):

        f = df.iloc[i-l:i]['sales_value'].values # ultimas 24h
        y = df.iloc[i:i+h]['sales_value'].values # proximas 24h

        X.append(f)
        Y.append(y)

    X = np.array(X)
    Y = np.array(Y)

    X = np.expand_dims(X, -1)
    X = np.swapaxes(X, 0, 1)
    
    return torch.from_numpy(X).float(), torch.from_numpy(Y).float()

In [6]:
train.head()

,date,sales_value
0,2020-02-01 00:00:00,28.60
1,2020-02-01 01:00:00,122.84
2,2020-02-01 02:00:00,112.87
3,2020-02-01 03:00:00,357.20
4,2020-02-01 04:00:00,642.56


In [7]:
X, Y   = prep_seqs(train)
Xt, Yt = prep_seqs(val)

In [8]:
X.shape, Y.shape

(torch.Size([24, 479, 1]), torch.Size([479, 24]))

In [9]:
mean_train = X.mean(1, keepdims=True)
std_train  = X.std(1,  unbiased=False, keepdims=True)

meanY_train = Y.mean(0, keepdims=True)
stdY_train  = Y.std(0, unbiased=False, keepdims=True)

In [10]:
X -= mean_train
X /= std_train

Xt -= mean_train
Xt /= std_train

Y -= meanY_train
Y /= stdY_train

In [11]:
class LSTMRNN(nn.Module):
    
    def __init__(self, hidden_size):

        super(LSTMRNN, self).__init__()

        self.hidden_size = hidden_size
        self.lstm        = nn.LSTM(1, self.hidden_size)
        self.out         = nn.Linear(self.hidden_size, 24)
        
    def forward(self, input):

        print("Input: ", input.shape)
        output, h = self.lstm(input)
        print("Output (hidden) da LSTM: ", output.shape)
        output = output[-1, :, :]
        print("Output (hidden state) do ultimo passo: ", output.shape)
        # print(torch.allclose(output, h[0]))
        # assert torch.allclose(output, h[0])
        
        output = self.out(output)
        print("Output da RNN: ", output.shape)
        return output.squeeze()
    
    

In [12]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)

lstm = LSTMRNN(100)
print(lstm)

LSTMRNN(
  (lstm): LSTM(1, 100)
  (out): Linear(in_features=100, out_features=24, bias=True)
)


In [13]:
criterion      = nn.L1Loss()
lstm_optimizer = optim.Adam(lstm.parameters(), lr=1e-2)

In [14]:
for i in range(100):

    lstm.zero_grad()

    o = lstm(X)
    loss = criterion(o, Y)
    loss.backward()
    lstm_optimizer.step()
    print(loss.item())
    print()
    
    p = lstm(Xt)
    p = p.detach()
    
    p *= stdY_train
    p += meanY_train
    p = p.numpy().squeeze()
    # break
    

print(f'Erro: {mean_absolute_error(Yt.numpy(), p)}')

Input:  torch.Size([24, 479, 1])
Output (hidden) da LSTM:  torch.Size([24, 479, 100])
Output (hidden state) do ultimo passo:  torch.Size([479, 100])
Output da RNN:  torch.Size([479, 24])
0.8392308354377747

Input:  torch.Size([24, 121, 1])
Output (hidden) da LSTM:  torch.Size([24, 121, 100])
Output (hidden state) do ultimo passo:  torch.Size([121, 100])
Output da RNN:  torch.Size([121, 24])
Input:  torch.Size([24, 479, 1])
Output (hidden) da LSTM:  torch.Size([24, 479, 100])
Output (hidden state) do ultimo passo:  torch.Size([479, 100])
Output da RNN:  torch.Size([479, 24])
0.820993185043335

Input:  torch.Size([24, 121, 1])
Output (hidden) da LSTM:  torch.Size([24, 121, 100])
Output (hidden state) do ultimo passo:  torch.Size([121, 100])
Output da RNN:  torch.Size([121, 24])
Input:  torch.Size([24, 479, 1])
Output (hidden) da LSTM:  torch.Size([24, 479, 100])
Output (hidden state) do ultimo passo:  torch.Size([479, 100])
Output da RNN:  torch.Size([479, 24])
0.7910199761390686

Input: